In [2]:
library(package = 'tidyverse', quietly = TRUE)
library(package = 'hipathia', quietly = TRUE) # hipathia package
library(package = 'reticulate', quietly = TRUE) # embedding a Python session within R session

# Enabling Python environment
# use_python("/opt/anaconda3/bin/python")
use_virtualenv("gpu_env")

setwd('/home/pgundogdu/projects/doc_p1_NN_with_pathways/')
getwd()

[1] "/home/pgundogdu/projects/doc_p1_NN_with_pathways"

In [3]:
# importing py script
src_script<-py_run_file('./config/settings.py')
po <- py_run_file(file.path(src_script$DIR_CONFIG, 'path_operations.py'))

# # defining species of
species = 'hsa'

In [4]:
# creating output folders
output_folder_raw = po$define_folder(file.path(loaded_script$DIR_DATA_RAW, species, 'hipathia'))
output_folder_processed = po$define_folder(file.path(loaded_script$DIR_DATA_PROCESSED, species, 'hipathia'))
output_folder_detail = po$define_folder(file.path(loaded_script$DIR_DATA_PROCESSED, species, 'hipathia', 'details'))

# importing pathway information file
dataset_hp <- read.table(file.path(src_script$DIR_DATA_PROCESSED, species, 'hipathia', 'pathway_ids_and_names.csv'), sep=',', header=TRUE)

# loading pathways 
pathways <- load_pathways(species = species)

In [7]:
# Loaded 146 pathways
paste0('Number of pathway      , ', length(unique(df_pathways$path.id)) )
paste0('Number of sub-pathways , ', nrow(df_pathways))
# [1] "Number of pathway      , 146"
# [1] "Number of sub-pathways , 6826"

print(' ')
print('SIGNALING PATHWAYS WITHOUT CANCER/DISEASE')
df_pathways = merge(dataset_hp, df_pathways)
print(paste0('Number of pathway      , ', nrow(dataset_hp)))
print(paste0('Number of sub-pathways , ', nrow(df_pathways)))
list_path_id = unique(df_pathways$path.id)
# [1] "SIGNALING PATHWAYS WITHOUT CANCER/DISEASE"
# [1] "Number of pathway      , 93"
# [1] "Number of sub-pathways , 4502"

# print(list_path_id[1])
# list_path_id

[1] "ALL SIGNALING PATHWAYS"
[1] "Number of pathway      , 146"
[1] "Number of sub-pathways , 6826"
[1] " "
[1] "SIGNALING PATHWAYS WITHOUT CANCER/DISEASE"
[1] "Number of pathway      , 93"
[1] "Number of sub-pathways , 4502"


In [ ]:
# GETTING GENE LIST

# Creating empty data frame to store genes
df_gene <- data.frame(pathways$all.genes)
# updating column name as 'entrez'
colnames(df_gene) = c('entrez')
l_gene_final<-c()

# for (all_pathways_ in c(1:length(list_path_id))) {
for (all_pathways_ in c(1:2)) {
    
    df_merge = df_gene
    l_main<-c()
    
#     getting sub-pathway
    length_subpathways = length(pathways$pathigraphs[[list_path_id[all_pathways_]]]$effector.subgraphs)
#     number of sub-pathway in 
    print(length_subpathways)
#     details for sub-pathway
    for (sub_pathways_ in c(1:length_subpathways)) {
        l_sub<-c()
        genes_circuits = V(pathways$pathigraphs[[list_path_id[all_pathways_]]]$effector.subgraphs[[sub_pathways_]])$genesList
        sub_path_name = names(pathways$pathigraphs[[list_path_id[all_pathways_]]]$effector.subgraphs[sub_pathways_][1])
#         name of sub-patway
        print(sub_path_name)
        for (sub_circuits in c(1:length(genes_circuits))){
            for (genes_ in c(1:length(genes_circuits[sub_circuits][1][[1]]))){
                gene_value = genes_circuits[sub_circuits][1][[1]][genes_]
                if (!is.na(gene_value) && gene_value != '/' && (gene_value == 'NA') == FALSE){
                    l_sub <-append(l_sub,gene_value)
                }
            }
            l_sub = unique(l_sub)
        }
#             Combining genes obtaining from sub-pathways
        if (length(l_sub) != 0) {
            l_main<-append(l_main, l_sub)
            df_temp <- data.frame(l_sub, 1)
            names(df_temp) = c('entrez', sub_path_name)
            print(df_temp)
        }
#         Inner join
        df_merge = merge(df_temp,df_merge, by='entrez', all=T)
#         print(df_merge)
    }
    l_main = unique(l_main)
#     Assigning all NA's as 0
#     df_merge[s.na(df_merge)] <- 0
    df_merge[!is.na(df_merge)] 
#     x[!is.na(x)]
    indices_genes <- as.vector(which(df_merge$entrez %in% l_main, arr.ind = TRUE))
#     1L * df_merge$entrez %in% c('info', l_main)
    df_path_genes <- (df_merge[c(indices_genes), ])
    rownames(df_path_genes) <- 1:nrow(df_path_genes)
    l_gene_final = append(l_gene_final, l_main)
#     print(length(l_gene_final))
#     Exporting gene set of each pathways (93 txt file for using pathways for hsa)
#     write.table(df_path_genes, paste0(output_folder_detail,list_path_id[all_pathways_],'_gene_list.txt'), sep=',', row.names=FALSE)
}
    
# write.table(df_gene$entrez[(df_gene$entrez) != 'NA'], paste0(output_folder_raw,'gene_list_all.csv'),sep=',',row.names = FALSE)
# write.table(unique(l_gene_final), paste0(output_folder_processed,'gene_list.csv'),sep=',', row.names = FALSE)

# print(paste0('THE GENES LIST EXPORTED!!! - ', output_folder,'gene_list_all.csv and', output_folder,'gene_list.csv'))